In [ ]:
#we shall import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)

In [ ]:
#importing the dataset
df1=pd.read_csv('/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
df1.head()                

In [ ]:
#we will check what is in dataset
df1.shape

In [ ]:
#now we will group the dataset by area type
df1.groupby('area_type')['area_type'].agg('count')

In [ ]:
# we will drop certain column from the datafreme, like availability, society,balcony,availability 
#as they are not important in predicting the price
df2=df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.head()

In [ ]:
#now we will find the number of null values in the dataframe
df2.isnull().sum()

In [ ]:
#now we will drop the null values in the dataframe as they are very less
df3=df2.dropna()
df3.isnull().sum()

In [ ]:
df3.shape

In [ ]:
#the size colums has both categorical as well as numerical data, we will see unique values
df3['size'].unique()

In [ ]:
#now we will create a separate column BHK containing number of rooms
df3['bhk']=df3['size'].apply(lambda x:int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:

df3['bhk'].unique()
#here rooms like 27,43 and others seems astronomical

In [ ]:
#now we will find the outliers
df3[df3.bhk>20]
#the outplut seems like an error

In [ ]:
#now we will find the unique values in the total_sqft column
df3.total_sqft.unique()

#the total_sqft has range values, this is a hindrance, we will try to remove that in future codes

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
#now we will see slice the total_sqft column based on the float
df3[~df3['total_sqft'].apply(is_float)].head(10)

In [ ]:
#now we will convert float into range by averaging the sqft ranges


In [ ]:
def convert_sqft_to_num(x):
    tokens=x.split('-')
    if len(tokens)==2:
        return(float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None
    

In [ ]:
#now we will taste the function
convert_sqft_to_num('2166')

In [ ]:
convert_sqft_to_num('2100-3000')
#here the output is average of the range

In [ ]:
#now we will apply this function to the sqft column and creat a new dataframe
df4=df3.copy()
df4['total_sqft']=df4['total_sqft'].apply(convert_sqft_to_num)
df4.head()

In [ ]:
#now we will start with feature engineering techniques and dimensionality reduction techniques
df5=df4.copy()
#now we will create price per sqft
df5['price_per_sqft']=df5['price']*100000/df5['total_sqft']
df5.head()

In [ ]:
#now we will explore location column
len(df5.location.unique())

In [ ]:
# there are 1304 features, means this is too much feature
#this is called dimensionality curse
#we shall reduce this dimensionality as low as possible
df5.location=df5.location.apply(lambda x:x.strip())
location_stats=df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
#now we will find the location with less than 10 datapoints
len(location_stats[location_stats<=10])

In [ ]:
location_stats_less_than_10=location_stats[location_stats<=10]
location_stats_less_than_10

In [ ]:
len(df5.location.unique())

In [ ]:
df5.location=df5.location.apply(lambda x: 'other'if x in location_stats_less_than_10 else x )
len(df5.location.unique())

In [ ]:
#all the locations with less than 10 properties are converted to other location 
df5.head(10)

In [ ]:
#now we will start outlier detections and removal
df5[df5.total_sqft/df5.bhk<300].head()

In [ ]:
df5.shape

In [ ]:
#now we will negate the criteria from above dataframe
df6=df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

In [ ]:
#now we will chaek the outliers in price per sqft
df6.price_per_sqft.describe()

In [ ]:
# we will filter the data beyond (mean +1std deviation).
def remove_pps_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-st))&(subdf.price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7=remove_pps_outliers(df6)
df7.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2=df[(df.location==location)&(df.bhk==2)]
    bhk3=df[(df.location==location)&(df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+',color='green',label='3 BHK',s=50)
    plt.xlabel('total square feet area')
    plt.ylabel('price')
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df7,'Hebbal')

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
# df8 = df7.copy()
df8.shape

In [ ]:
plot_scatter_chart(df8,'Hebbal')

In [ ]:
#now we will pot a histogram with price per squarefeet
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel('price per squarefoot')
plt.ylabel('count')


In [ ]:
#now we will explore bathroom feature
df8.bath.unique()

In [ ]:
df8[df8.bath>10]

In [ ]:
plt.hist(df8.bath,rwidth=0.8)
plt.xlabel('number of bathrooms')
plt.ylabel('count')

In [ ]:
#now we will remove any time we have bathrooms greater than number of bedrooms +2 is an outlier
df8[df8.bath>df8.bhk+2]


In [ ]:
df9=df8[df8.bath<df8.bhk+2]
df9.shape

In [ ]:
#now we will prepare our dataframe for ML training
#we shall remove price per squarefoot and size
df10=df9.drop(['size','price_per_sqft'],axis='columns')
df10.head(3)

In [ ]:
#we will perform one hot encoding for location column
dummies=pd.get_dummies(df10.location)
dummies.head()

In [ ]:
df11=pd.concat([df10,dummies],axis='columns')

In [ ]:
#to avoid dummy trap, we will drop last column
df11=df11.drop(['other'],axis='columns')

In [ ]:
df11.head()

In [ ]:
df12=df11.drop('location',axis='columns')
df12.head(2)

In [ ]:
df12.shape

In [ ]:
#will define dependent variable for training
X=df12.drop('price',axis='columns')
X.head()

In [ ]:
#will define independent variable for training
y=df12.price
y.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=10)


In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

In [ ]:
#now we will perform k fold cross validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),X,y,cv=cv)

In [ ]:
#now we will perform grid search cv, here we will try multiple model and select best one with good score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)
    


In [ ]:
X.columns

In [ ]:
np.where(X.columns=='2nd Phase Judicial Layout')[0][0]

In [ ]:
#here the winner is Linear Regression
#Now we will use linear regression for property price prediction
def predict_price(location,sqft,bath,bhk):
    loc_index=np.where(X.columns==location)[0][0]
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=bhk
    if loc_index >=0:
        x[loc_index]=1
        
    return lr_clf.predict([x])[0]

# Predictions

In [ ]:
predict_price('1st Phase JP Nagar',1000,2,2)

In [ ]:
predict_price('1st Phase JP Nagar',1000,3,3)

# Now you can tryout with your imagination